In [91]:
# !pip install pypdf3
# !pip install tabula-py
# !pip install camelot-py

import pandas as pd
import numpy as np
# import camelot as camel
from PyPDF3 import PdfFileReader 
from tabula import read_pdf 
# from tabulate import tabulate

from my_modules import my_functions as mybib
from my_modules import project_functions as pr


%load_ext autoreload
%autoreload 2


# https://www.youtube.com/watch?v=702lkQbZx50 - for tabula
# https://stackoverflow.com/questions/45457054/tabula-extract-tables-by-area-coordinates - for tabula 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [92]:
import os 
import re


In [93]:
def transform_number_under_10(x):
    if x < 10:
        x = '0' + str(x)
        return x
    else:
        x = str(x)
        return x

In [94]:
#FIXME: close pdf file when error occurs
# https://stackoverflow.com/questions/43259717/progress-bar-for-a-for-loop-in-python-script
import time 
from rich.progress import track
from tqdm import tqdm

def extract_all_pages_from_pdf(path, file_name):
    # open path as binary / rb = read binary -> used for pdf files (https://www.youtube.com/watch?v=OZo2HxoIOtw)
            
    pdf = PdfFileReader(open(path, 'rb'))
    last_page = pdf.getNumPages()
    last_page += 1

    # defining page range for import 
    [i for i in range(0, 15)]
    page_range = [i for i in range(5, last_page)]
    # print(page_range)

    # prepare column name filler
    column_names = [i for i in range(0, 14)]
    # print(column_names)

    # create empty dataframe
    adress_data = pd.DataFrame(columns = column_names)
    # display(adress_data)
    
    # extract district information 
    #TODO: nochmal prüfen und vielleicht optimieren
    district = [re.findall(r'(?<=adr_)(.*?)(?=_\d{4}\.pdf)', file_name)]
    print(district[0][0])

    for page in tqdm(page_range):
        
        import_data = read_pdf(path, pages = page, encoding = 'ISO-8859-1', stream = True, area = [175, 33, 783, 520], guess = True, pandas_options={'header': None})
        table_df = import_data[0]

        
        table_df = import_data[0]
        columns_len = len(table_df.columns)

        if columns_len < 14:
            if table_df.iloc[:, 3].dtype in ['int64', 'int32', 'float64', 'float32']:
                table_df.insert(3, 'm1', np.nan)

            if table_df.iloc[:, 6].dtype in ['int64', 'int32', 'float64', 'float32']:
                table_df.insert(5, 'm2', np.nan)
            
            
            column_length = len(table_df.columns)

            
            if column_length < 14:
                table_df['m3'] = np.nan
                # display(table_df)
            
            
            table_df.columns = column_names
            table_df['14'] = str(district[0][0])
            
            adress_data = pd.concat([adress_data, table_df], ignore_index=True , axis=0)
            
        # time.sleep(0.5)
        # columns_to_transform = [7,8,9,10]
        
        # for col in columns_to_transform:
        #     adress_data[col] = adress_data[col].apply(lambda x: transform_number_under_10(x))
        
        # print(f'page: {page} -> {adress_data.shape[0]}')
        # display(adress_data.head(10))

    return adress_data, district[0][0]



In [95]:



def get_adress_data_by_year(year):
    
    
    list_of_files = os.listdir(f'../data/input/berlin_adresses/{year}')
    list_of_files
        
    
    
    for file_name in list_of_files:
        path = f'../data/input/berlin_adresses/{year}/{file_name}'
        print(path)
        
        dataset, name = extract_all_pages_from_pdf(path, file_name)
        
        dataset.to_csv(f'../data/output/temp_adress_data/{name}-{year}.csv', index = False)
        display(dataset.head(5))
        print(dataset.shape)
    




In [96]:
# get_adress_data_by_year(2020)

In [97]:
get_adress_data_by_year(2023)

../data/input/berlin_adresses/2023/adr_charlottenburg-wilmdersdorf_2023.pdf
[['charlottenburg-wilmdersdorf']]


100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [01:36<00:00,  1.94s/it]


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,1,Aachener Straße,1,NaN,45,NaN,U,4,50,11,48,Wilmd,10713,NaN,charlottenburg-wilmdersdorf
1,1,Aachener Straße,2,NaN,44,NaN,G,4,50,11,48,Wilmd,10713,NaN,charlottenburg-wilmdersdorf
2,5,Abbestraße,1,NaN,19,NaN,U,4,30,6,22,Charl,10587,NaN,charlottenburg-wilmdersdorf
3,5,Abbestraße,2,NaN,12,NaN,G,4,30,6,22,Charl,10587,NaN,charlottenburg-wilmdersdorf
4,8793,Adam-von-Trott-Straße,1,NaN,7,NaN,U,4,10,1,1,CharN,13627,NaN,charlottenburg-wilmdersdorf


(1701, 15)
../data/input/berlin_adresses/2023/adr_friedrichshain-kreuzberg_2023.pdf
[['friedrichshain-kreuzberg']]


100%|██████████████████████████████████████████████████████████████████████████████████| 28/28 [00:41<00:00,  1.48s/it]


...

100%|██████████████████████████████████████████████████████████████████████████████████| 33/33 [00:51<00:00,  1.55s/it]


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,44843,Alt-Friedrichsfelde,87,NaN,127,A,U,11,40,10,32,Frife,10315,NaN,lichtenberg
1,44843,Alt-Friedrichsfelde,88,NaN,126,NaN,G,11,40,10,32,Frife,10315,NaN,lichtenberg
2,10848,Alzeyweg,2,NaN,44,NaN,G,11,30,7,25,Liber,10365,NaN,lichtenberg
3,10875,Am Ahornweg,12,NaN,84,NaN,G,11,10,1,1,Wartb,13059,NaN,lichtenberg
4,10875,Am Ahornweg,13,NaN,85,NaN,U,11,10,1,1,Wartb,13059,NaN,lichtenberg


(1016, 15)
../data/input/berlin_adresses/2023/adr_marzahn-hellersdorf_2023.pdf
[['marzahn-hellersdorf']]


 10%|████████                                                                           | 4/41 [00:07<01:12,  1.96s/it]


KeyboardInterrupt: 

In [ ]:
# file_name = 'adr_friedrichshain-kreuzberg_2020.pdf'

# district = [re.findall(r'(?<=adr_)(.*?)(?=_\d{4}\.pdf)', file_name)]
# district

In [ ]:
# path = '../data/input/berlin_adresses/2023/adr_friedrichshain-kreuzberg_2023.pdf'

# extract_all_pages_from_pdf(path, 'adr_friedrichshain-kreuzberg_2023.pdf')